# Modeling - Vanilla Algorithms Multinomial logistic regression
Author: P.C.O. Maseland <br>
Date: 03 December 2021 <br>

In [1]:
# !!! RUN !!!
#Import packages/libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

#Linear Regression
from sklearn.linear_model import LogisticRegression

%matplotlib inline

### Load Correct datafile

**prepared accelerometer, gyroscope and magnetometer csv**

In [2]:
# !!! RUN !!!
#local jupyter notebook path
path ="Totalset+slopes+mean+sd.csv"
df = pd.read_csv(path)

In [3]:
# !!! RUN !!!
#reorder such that labels are at the back
df = df[[c for c in df if c not in ['Subject ID','Activity']] 
       + ['Subject ID','Activity']]

**Activity coding**
- 'climbingdown' {0} 
- 'climbingup' {1} 
- 'jumping' {2} 
- 'lying' {3} 
- 'running' {4} 
- 'sitting' {5}
- 'standing' {6} 
- 'walking' {7}]

### Remove/Convert statistical NaN values
Since we calculated our rolled statistics over a nr of timesteps. The first nr of timesteps have a NaN, these are therefore useless. On the whole set we removed thereby 24336 rows, which is just a small fraction of 0.4% of the whole.

In [4]:
# !!! RUN !!!
df = df.dropna()

### Full set copy

In [5]:
# !!! RUN !!!
#create copy full set
fullset = df
print(fullset.shape)

#drop subject ID
fullset_without_subject = fullset.drop(columns=['Subject ID'])

#shuffle fullset
fullset_without_subject = fullset_without_subject.sample(frac=1)

(6059664, 86)


### Make new subset
With only time-base statistical values

In [6]:
# !!! RUN !!!
statistical_cols = [col for col in fullset_without_subject.columns if ('mean' in col 
                                                                      or 'sd' in col
                                                                      or 'Activity' in col)]
statistical_cols

new_subset = fullset_without_subject[statistical_cols]
new_subset.shape

(6059664, 43)

# Model set up - Multinomial Logistical Regression modelling

#### best grid

In [7]:
#best model
best_grid = LogisticRegression(multi_class='multinomial', 
                            solver='lbfgs',
                            penalty='l2', C=0.325, max_iter=40, 
                            class_weight='balanced')

### Automated Functions

In [8]:
# !!! RUN !!!
def dataset_prep(dataset, filters):
    filterscolls = [col for col in dataset.columns if (filters in col or 'Activity' in col)]
    outputname = dataset[filterscolls]
    
    return outputname

In [9]:
# !!! RUN !!!
def multivar_log(outputname):
    X = dataset.iloc[:,:-1]
    y = dataset['Activity']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40, stratify=y)
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)
    
    #model
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=1.0, class_weight='balanced')
    #fit model
    model.fit(X_train, y_train)
    #make prediction and evaluation
    y_pred = model.predict(X_test)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    return model, X_train
    

In [10]:
# !!! RUN !!!
def multivar_log_new(gridmodel, dataset):
    X = dataset.iloc[:,:-1]
    y = dataset['Activity']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40, stratify=y)
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)
    
    #model
    model = gridmodel
    
    #fit model
    model.fit(X_train, y_train)
    
    y_pred_test = model.predict(X_test)
    y_pred_rnd = np.around(y_pred_test, 0)
    
    # Calculate the absolute errors
    errors = abs(y_pred_rnd - y_test)

    # Print out the mean absolute error (MAE)
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
    
    #Evaluating Matrix
    print(confusion_matrix(y_test, y_pred_rnd))
    print(classification_report(y_test, y_pred_rnd))
    
    return model, X_train
    

In [11]:
def multivar_log_importance(model, X_train):
    # get importance
    importance = model.coef_

    from matplotlib import pyplot
    avg_importance = sum(importance)/len(importance)
    pyplot.bar([x for x in range(len(avg_importance))], avg_importance)
    pyplot.show()

    print(X_train.columns)

## Single Sensor position - Subq 1
Slope score low on feature importance score

Positions: <br>
Head <br>
Upperarm <br>
Forearm <br>
Chest <br>
Waist <br>
Thigh <br>
Shin <br>

Different Sensors: <br>
acc (mean, sd) <br>
gyr (mean, sd) <br>
mag (mean, sd) <br>
acc + mag (mean, sd) <br>
acc + gyr (mean, sd) <br>
gyr + mag (mean, sd) <br>
acc + gyr + mag (mean, sd) <br>

#### Single sensor HEAD

In [27]:
#generate dataset samples
head_acc_subset = dataset_prep(new_subset, 'head_acc')
head_gyr_subset = dataset_prep(new_subset, 'head_gyr')
head_mag_subset = dataset_prep(new_subset, 'head_mag')
filterscolls1 = [col for col in new_subset.columns if ('head_acc' in col 
                                                   or 'head_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('head_acc' in col 
                                                   or 'head_mag' in col 
                                                   or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('head_mag' in col 
                                                   or 'head_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls4 = [col for col in new_subset.columns if ('head_mag' in col
                                                      or 'head_gyr' in col
                                                      or 'head_acc' in col
                                                      or 'Activity' in col)]

head_accgyr_subset = new_subset[filterscolls1] 
head_accmag_subset = new_subset[filterscolls2]  
head_maggyr_subset = new_subset[filterscolls3]
head_accmaggyr_subset = new_subset[filterscolls4]

In [28]:
#Single Head - acc
head_acc, head_acc_X = multivar_log_new(best_grid, head_acc_subset)

#Single Head - gyr
head_gyr, head_gyr_X = multivar_log_new(best_grid, head_gyr_subset)

#Single Head - mag
head_mag, head_mag_X = multivar_log_new(best_grid, head_mag_subset)

#Single Head - acc + gyr
head_accgyr, head_accgyr_X = multivar_log_new(best_grid, head_accgyr_subset)

#Single Head - acc + mag
head_accmag, head_accmag_X = multivar_log_new(best_grid, head_accmag_subset)

#Single Head - gyr + mag
head_maggyr, head_maggyr_X = multivar_log_new(best_grid, head_maggyr_subset)

#Single Head - acc + gyr + mag
head_accmaggyr, head_accmaggyr_X = multivar_log_new(best_grid, head_accmaggyr_subset)

(4847731, 2)
(4847731,)
(1211933, 2)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.06 degrees.
[[ 21974  47054   5299    435  20406     68   5807  19473]
 [ 14761  76136    923   5539   5570    348  14489  20080]
 [   851   2306  10958    268   4089     33    665   1547]
 [   166   3085     19 167006     60   1242  12281    202]
 [ 13341  11393  80070  12799  57467    833  13584   8319]
 [    41  13421      4 116415     10   3713  49919    139]
 [   110  17163      1 100653     20   4687  58956    280]
 [ 30885  82955   5419    981  23266    144  12942  28863]]
              precision    recall  f1-score   support

           0       0.27      0.18      0.22    120516
           1       0.30      0.55      0.39    137846
           2       0.11      0.53      0.18     20717
           3       0.41      0.91      0.57    184061
           4       0.52      0.29      0.37    197806
           5       0.34      0.02      0.04    183662
           6       0.35      0.32      0.34    181870
           7       0.37      0.16      0.22    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.02 degrees.
[[ 10217  34574  20521   1363  10043   7974  12502  23322]
 [  7936  53255   7980   6109   7264  10117  24233  20952]
 [  1189   1528  10453     59   3149   1012    606   2721]
 [   281   1900    367 166530    410  10520   2533   1520]
 [ 16920  36335  64635  12700  37951   7954   6870  14441]
 [  1916  10446   3863 105549   2693  38989  10410   9796]
 [  2694  15516   4770  79663   3148  42167  20238  13674]
 [ 12392  42765  17623   4005   6264  18466  36393  47547]]
              precision    recall  f1-score   support

           0       0.19      0.08      0.12    120516
           1       0.27      0.39      0.32    137846
           2       0.08      0.50      0.14     20717
           3       0.44      0.90      0.59    184061
           4       0.54      0.19      0.28    197806
           5       0.28      0.21      0.24    183662
           6       0.18      0.11      0.14    181870
           7       0.35      0.26      0.30    185455

    

...

Mean Absolute Error: 1.98 degrees.
[[ 29791  32321   6515   4550  15394   3806      0  28139]
 [ 17035  61126   1776  17568   3361   4304      1  32675]
 [  2048    762   7477   1026   6602   1243      0   1559]
 [   144   3044     33  70849     41 109565      2    383]
 [ 27122   9041  49619   6316  77804  20492      3   7409]
 [   805  11196      6  33852      4 137522      5    272]
 [   797  18286      2  44000     10 118193      8    574]
 [ 37170  61793   7514  11779  16381   4931      1  45886]]
              precision    recall  f1-score   support

           0       0.26      0.25      0.25    120516
           1       0.31      0.44      0.36    137846
           2       0.10      0.36      0.16     20717
           3       0.37      0.38      0.38    184061
           4       0.65      0.39      0.49    197806
           5       0.34      0.75      0.47    183662
           6       0.40      0.00      0.00    181870
           7       0.39      0.25      0.30    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.01 degrees.
[[ 15746  32419  16296   1771  13643   8476  16016  16149]
 [  9112  60693   5697   7965  10199   5490  19097  19593]
 [  1408   1789   9941    498   3061   1152   1700   1168]
 [   366   2716    240 100213    513  72514   6882    617]
 [ 14812  29358  61737  10055  47134   6886  13246  14578]
 [  6475   8246   3519  76705   3070  64129  16842   4676]
 [  5338  17925   3839  54512   4199  50000  38229   7828]
 [ 12682  49989  13176   5266  10184  11666  45595  36897]]
              precision    recall  f1-score   support

           0       0.24      0.13      0.17    120516
           1       0.30      0.44      0.36    137846
           2       0.09      0.48      0.15     20717
           3       0.39      0.54      0.45    184061
           4       0.51      0.24      0.33    197806
           5       0.29      0.35      0.32    183662
           6       0.24      0.21      0.23    181870
           7       0.36      0.20      0.26    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.91 degrees.
[[ 29771  33313   8083   3079  19069   2104    697  24400]
 [ 20756  63560   1979  12805   4807   1369   1841  30729]
 [  1362   1229  10209   1083   5591    891     55    297]
 [   304   3239     73  70306     34 108330   1588    187]
 [ 18373  11720  67022   4036  73250  13830   6924   2651]
 [   390  17798     13  31962      3 119151  13964    381]
 [   502  26584     35  33807     10  90609  29713    610]
 [ 33115  59132   5930   7762  22422   2216   2188  52690]]
              precision    recall  f1-score   support

           0       0.28      0.25      0.26    120516
           1       0.29      0.46      0.36    137846
           2       0.11      0.49      0.18     20717
           3       0.43      0.38      0.40    184061
           4       0.59      0.37      0.45    197806
           5       0.35      0.65      0.46    183662
           6       0.52      0.16      0.25    181870
           7       0.47      0.28      0.35    185455

    

#### Single sensor Forearm

In [35]:
#create subsets for forearm
forearm_acc_subset = dataset_prep(new_subset, 'forearm_acc')
forearm_gyr_subset = dataset_prep(new_subset, 'forearm_gyr')
forearm_mag_subset = dataset_prep(new_subset, 'forearm_mag')
filterscolls1 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                   or 'forearm_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                   or 'forearm_mag' in col 
                                                   or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('forearm_mag' in col 
                                                   or 'forearm_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls4 = [col for col in new_subset.columns if ('forearm_mag' in col
                                                      or 'forearm_gyr' in col
                                                      or 'forearm_acc' in col
                                                      or 'Activity' in col)]
forearm_accgyr_subset = new_subset[filterscolls1] 
forearm_accmag_subset = new_subset[filterscolls2]
forearm_maggyr_subset = new_subset[filterscolls3]
forearm_accmaggyr_subset = new_subset[filterscolls4] 

In [36]:
#run all models for single sensor
forearm_acc, forearm_acc_X = multivar_log_new(best_grid, forearm_acc_subset)
forearm_gyr, forearm_gyr_X = multivar_log_new(best_grid, forearm_gyr_subset)
forearm_mag, forearm_mag_X = multivar_log_new(best_grid, forearm_mag_subset)

(4847731, 2)
(4847731,)
(1211933, 2)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.12 degrees.
[[ 19275  39414  12130    954  21576    516   8604  18047]
 [ 16125  60215   3883   6829   8575   1594  19237  21388]
 [  1860   3162   8957     33   4634    700    158   1213]
 [   417   3055    131 168993    160   3081   7723    501]
 [ 17624  22568  49822  14566  67749   3266   9745  12466]
 [  1368  10390    642 138878    390   8278  22165   1551]
 [  1050  12523    461 114504    479  12385  38964   1504]
 [ 30331  77018   7362   1629  14150    964  18656  35345]]
              precision    recall  f1-score   support

           0       0.22      0.16      0.18    120516
           1       0.26      0.44      0.33    137846
           2       0.11      0.43      0.17     20717
           3       0.38      0.92      0.54    184061
           4       0.58      0.34      0.43    197806
           5       0.27      0.05      0.08    183662
           6       0.31      0.21      0.25    181870
           7       0.38      0.19      0.25    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.95 degrees.
[[  5639  31042  31574    297  26289   1055  10294  14326]
 [  5748  48474  27019   4087  13515   2667  16742  19594]
 [   727   4068   9130      0   3308    291   2232    961]
 [   106   1217    912 172658    279   6189   2368    332]
 [  8594  13361  40540  11829  91869   7841   8131  15641]
 [   381   3794   4320 136066   1436  25020  11684    961]
 [   521   6609   5210  85001   2323  45270  35581   1355]
 [  9524  63304  42803    194  18493   1785  15789  33563]]
              precision    recall  f1-score   support

           0       0.18      0.05      0.07    120516
           1       0.28      0.35      0.31    137846
           2       0.06      0.44      0.10     20717
           3       0.42      0.94      0.58    184061
           4       0.58      0.46      0.52    197806
           5       0.28      0.14      0.18    183662
           6       0.35      0.20      0.25    181870
           7       0.39      0.18      0.25    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.23 degrees.
[[ 4904 14097 19076 14525 24120 15625 26516  1653]
 [ 3479 20655 24794 15673 21480 30084 20261  1420]
 [  603  3111  6434   664  2092  4552  3049   212]
 [ 6318 13465 46688 29653 27968 21548 35958  2463]
 [ 6049 31016 31507 14812 42924 30745 37255  3498]
 [ 1575 39595 34590  3357  9163 80832 13150  1400]
 [ 7413 23655 19886 18237 29110 37552 43242  2775]
 [ 5539 36882 10432 11585 27642 50030 39768  3577]]
              precision    recall  f1-score   support

           0       0.14      0.04      0.06    120516
           1       0.11      0.15      0.13    137846
           2       0.03      0.31      0.06     20717
           3       0.27      0.16      0.20    184061
           4       0.23      0.22      0.22    197806
           5       0.30      0.44      0.36    183662
           6       0.20      0.24      0.22    181870
           7       0.21      0.02      0.04    185455

    accuracy                           0.19   1211933
   macro avg  

In [44]:
#run all models for combination sensor
forearm_accgyr, forearm_accgyr_X = multivar_log_new(best_grid, forearm_accgyr_subset)
forearm_accmag, forearm_accmag_X = multivar_log_new(best_grid, forearm_accmag_subset)
forearm_maggyr, forearm_maggyr_X = multivar_log_new(best_grid, forearm_maggyr_subset)
forearm_accmaggyr, forearm_accmaggyr_X = multivar_log_new(best_grid, forearm_accmaggyr_subset)

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.96 degrees.
[[ 29751  28675  22929    304  19591    104   3122  16040]
 [ 26200  57763  10217   3203   6080   1788   9378  23217]
 [  1563   1183  12857      0   3781    113    910    310]
 [   471   1834    365 170646     93   6367   3978    307]
 [ 29830  10275  41892   9238  86321   7939   8462   3849]
 [  2072   7149   1568 129517    206  24397  17916    837]
 [  3150  10196   1253  78491    206  43269  44529    776]
 [ 39727  69698  18780    192   9991    137   6369  40561]]
              precision    recall  f1-score   support

           0       0.22      0.25      0.23    120516
           1       0.31      0.42      0.36    137846
           2       0.12      0.62      0.20     20717
           3       0.44      0.93      0.59    184061
           4       0.68      0.44      0.53    197806
           5       0.29      0.13      0.18    183662
           6       0.47      0.24      0.32    181870
           7       0.47      0.22      0.30    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.98 degrees.
[[ 39062  14211  12752    662  23767  18597    876  10589]
 [ 39405  21526   3970    643  10273  46149    862  15018]
 [  3278    251  10020    196   4772    749    730    721]
 [  1002    844    140   3490    183 177962      1    439]
 [ 29302   6861  52657    473  68185  32025   3763   4540]
 [  3092   2697    672    310    492 174534     23   1842]
 [  2785   2885    514   2580    516 170786    182   1622]
 [ 66562  27486   7886     86  15306  45321    220  22588]]
              precision    recall  f1-score   support

           0       0.21      0.32      0.26    120516
           1       0.28      0.16      0.20    137846
           2       0.11      0.48      0.18     20717
           3       0.41      0.02      0.04    184061
           4       0.55      0.34      0.42    197806
           5       0.26      0.95      0.41    183662
           6       0.03      0.00      0.00    181870
           7       0.39      0.12      0.19    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.86 degrees.
[[  6536  21017  27174     54  28670    832   7592  28641]
 [  7468  32233  22182   1487  14665   4500  12707  42604]
 [   615   2495   7531      0   4327     24   2005   3720]
 [   103    859    707  71271    326 106766   2943   1086]
 [  9285  11154  33250   2564  98824  17440   6979  18310]
 [   364   2596   3573  35694   1480 123289  13027   3639]
 [   441   4877   4517  22947   2439 107350  32921   6378]
 [  9063  41840  33163     35  18708    519  13802  68325]]
              precision    recall  f1-score   support

           0       0.19      0.05      0.08    120516
           1       0.28      0.23      0.25    137846
           2       0.06      0.36      0.10     20717
           3       0.53      0.39      0.45    184061
           4       0.58      0.50      0.54    197806
           5       0.34      0.67      0.45    183662
           6       0.36      0.18      0.24    181870
           7       0.40      0.37      0.38    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.88 degrees.
[[ 31966  32088  17590    776  22537   1504   2126  11929]
 [ 27489  58540   8640   3851   7682   5283   6151  20210]
 [  1903   3161   9238   1009   4573      4    378    451]
 [   407   1962    317  69641    100 110299   1062    273]
 [ 31266  10045  35011   3107  91123  20632   2394   4228]
 [  1478   7863   1343  33084    223 135417   3386    868]
 [  2054  12430   1149  19635    191 136138   9160   1113]
 [ 43975  71828  15786    701  10871   2322   5358  34614]]
              precision    recall  f1-score   support

           0       0.23      0.27      0.24    120516
           1       0.30      0.42      0.35    137846
           2       0.10      0.45      0.17     20717
           3       0.53      0.38      0.44    184061
           4       0.66      0.46      0.54    197806
           5       0.33      0.74      0.45    183662
           6       0.31      0.05      0.09    181870
           7       0.47      0.19      0.27    185455

    

#### Single sensor Waist

In [45]:
#create subsets for Waist
Waist_acc_subset = dataset_prep(new_subset, 'waist_acc')
Waist_gyr_subset = dataset_prep(new_subset, 'waist_gyr')
Waist_mag_subset = dataset_prep(new_subset, 'waist_mag')
filterscolls1 = [col for col in new_subset.columns if ('waist_acc' in col 
                                                   or 'waist_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('waist_acc' in col 
                                                   or 'waist_mag' in col 
                                                   or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('waist_mag' in col 
                                                   or 'waist_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls4 = [col for col in new_subset.columns if ('waist_mag' in col
                                                      or 'waist_gyr' in col
                                                      or 'waist_acc' in col
                                                      or 'Activity' in col)]
Waist_accgyr_subset = new_subset[filterscolls1] 
Waist_accmag_subset = new_subset[filterscolls2]
Waist_maggyr_subset = new_subset[filterscolls3]
Waist_accmaggyr_subset = new_subset[filterscolls4] 

In [46]:
#run all models for single sensor
Waist_acc, Waist_acc_X = multivar_log_new(best_grid, Waist_acc_subset)
Waist_gyr, Waist_gyr_X = multivar_log_new(best_grid, Waist_gyr_subset)
Waist_mag, Waist_mag_X = multivar_log_new(best_grid, Waist_mag_subset)

(4847731, 2)
(4847731,)
(1211933, 2)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.9 degrees.
[[ 20900  42966  18484    226  20746    194   2635  14365]
 [ 15637  72033   6233    677   8567   6078  10236  18385]
 [  1029   1971  11357      0   4488     29    670   1173]
 [    86   3708     20      5     18 155731  24363    130]
 [ 12674  13158  71027      6  60502  10185  17228  13026]
 [    26   3187      6      1      7 144245  36141     49]
 [    83  11359     30      9     11 101023  69220    135]
 [ 32181  78488  19883    285  16651    290   5490  32187]]
              precision    recall  f1-score   support

           0       0.25      0.17      0.21    120516
           1       0.32      0.52      0.40    137846
           2       0.09      0.55      0.15     20717
           3       0.00      0.00      0.00    184061
           4       0.55      0.31      0.39    197806
           5       0.35      0.79      0.48    183662
           6       0.42      0.38      0.40    181870
           7       0.41      0.17      0.24    185455

    a

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.89 degrees.
[[ 20197  30001  19065    108  14165     61   2334  34585]
 [ 15724  54363   8863    173   5567   2974  10720  39462]
 [  1991   1415   8413      1   3689      2    293   4913]
 [   132   1033     69 133063    203  22884  25692    985]
 [ 19148  10105  38595  11119  95400   4474  12784   6181]
 [    86   2694     23 117913     89  32323  29526   1008]
 [    82   6967     30  84890     77  29287  57645   2892]
 [ 19048  51201  18900    238   3726    129  10416  81797]]
              precision    recall  f1-score   support

           0       0.26      0.17      0.21    120516
           1       0.34      0.39      0.37    137846
           2       0.09      0.41      0.15     20717
           3       0.38      0.72      0.50    184061
           4       0.78      0.48      0.59    197806
           5       0.35      0.18      0.23    183662
           6       0.39      0.32      0.35    181870
           7       0.48      0.44      0.46    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.23 degrees.
[[ 45455      0   2637  33913  13390   1176   1275  22670]
 [ 50063      0   4343  27957  21790   1882   5404  26407]
 [  9239      0     52   2956   4880    238   1623   1729]
 [  5514      0    245 150775    923   4184  13513   8907]
 [ 68595      0   2544  29781  64504    121  22142  10119]
 [ 14586      0     40 121053   5646    539  33473   8325]
 [  8649      0    105 108547  11251   1804  39489  12025]
 [ 38948      0   1414  75320  26041   2262  20191  21279]]


C:\Users\31657\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\31657\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.19      0.38      0.25    120516
           1       0.00      0.00      0.00    137846
           2       0.00      0.00      0.00     20717
           3       0.27      0.82      0.41    184061
           4       0.43      0.33      0.37    197806
           5       0.04      0.00      0.01    183662
           6       0.29      0.22      0.25    181870
           7       0.19      0.11      0.14    185455

    accuracy                           0.27   1211933
   macro avg       0.18      0.23      0.18   1211933
weighted avg       0.21      0.27      0.21   1211933



C:\Users\31657\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
#run all models for combination sensor
Waist_accgyr, Waist_accgyr_X = multivar_log_new(best_grid, Waist_accgyr_subset)
Waist_accmag, Waist_accmag_X = multivar_log_new(best_grid, Waist_accmag_subset)
Waist_maggyr, Waist_maggyr_X = multivar_log_new(best_grid, Waist_maggyr_subset)
Waist_accmaggyr, Waist_accmaggyr_X = multivar_log_new(best_grid, Waist_accmaggyr_subset)

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.73 degrees.
[[ 27305  34609  13873      0  18305    180   1004  25240]
 [ 18853  60382   2943      2   7905   6120   5075  36566]
 [  1316   1205  13453      0   3275      2    328   1138]
 [    53   1156     28      0    222 168456  12108   2038]
 [  7092   5212  64861      0  88594  14724  12843   4480]
 [    28   1494      1      0    131 156250  24039   1719]
 [    32   2890      4      0     93 121046  51065   6740]
 [ 37671  54116   8744      0   5718    271   2987  75948]]
              precision    recall  f1-score   support

           0       0.30      0.23      0.26    120516
           1       0.37      0.44      0.40    137846
           2       0.13      0.65      0.22     20717
           3       0.00      0.00      0.00    184061
           4       0.71      0.45      0.55    197806
           5       0.33      0.85      0.48    183662
           6       0.47      0.28      0.35    181870
           7       0.49      0.41      0.45    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.05 degrees.
[[ 50059  29701  19435    444   8967   4014   3856   4040]
 [ 39112  50396   6508   2704   4325  20950   7158   6693]
 [  6954   1009   7445   1378   3425      5     57    444]
 [   254   1499     19   2354     11 176296   3595     33]
 [ 43172   7586  56601  18677  55041  12447    354   3928]
 [    81   1118      5  34543     14 147792     66     43]
 [   162   3136      7  45545     85 132448    363    124]
 [ 70095  54115  13894   4885  15129   9053   4182  14102]]
              precision    recall  f1-score   support

           0       0.24      0.42      0.30    120516
           1       0.34      0.37      0.35    137846
           2       0.07      0.36      0.12     20717
           3       0.02      0.01      0.02    184061
           4       0.63      0.28      0.39    197806
           5       0.29      0.80      0.43    183662
           6       0.02      0.00      0.00    181870
           7       0.48      0.08      0.13    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.9 degrees.
[[ 18217  12211  31439   1613  17142      8      7  39879]
 [ 18107  23390  17372  10158  15493      3    286  53037]
 [  1115    915   8847    251   5562      0      7   4020]
 [   253    326    118 181233    492      2    175   1462]
 [  8990   1717  47244  11634 105197      6  13329   9689]
 [   466   3502     64 156632    432    957  18324   3285]
 [   270   3150    131 124306    713   2318  34458  16524]
 [ 16084  13787  33189   6498  17899     19    289  97690]]
              precision    recall  f1-score   support

           0       0.29      0.15      0.20    120516
           1       0.40      0.17      0.24    137846
           2       0.06      0.43      0.11     20717
           3       0.37      0.98      0.54    184061
           4       0.65      0.53      0.58    197806
           5       0.29      0.01      0.01    183662
           6       0.52      0.19      0.28    181870
           7       0.43      0.53      0.48    185455

    a

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.81 degrees.
[[ 54154  24503  19570      9   8726    309   1903  11342]
 [ 43113  53583   5637   5375   3248   2989   6640  17261]
 [  4862   1773   9772      0   3467    715      0    128]
 [   246   1878     15 130221     40   2878  48467    316]
 [ 20070   9188  52897   1287  85366  19049   9348    601]
 [    80   1934      1  66179     10  30657  84602    199]
 [   116   4235     20  40899     31  48110  87890    569]
 [ 73506  52697  13985    171   7937   3943   3608  29608]]
              precision    recall  f1-score   support

           0       0.28      0.45      0.34    120516
           1       0.36      0.39      0.37    137846
           2       0.10      0.47      0.16     20717
           3       0.53      0.71      0.61    184061
           4       0.78      0.43      0.56    197806
           5       0.28      0.17      0.21    183662
           6       0.36      0.48      0.41    181870
           7       0.49      0.16      0.24    185455

    

#### Single sensor Thigh

In [48]:
#create subsets for Thigh
Thigh_acc_subset = dataset_prep(new_subset, 'thigh_acc')
Thigh_gyr_subset = dataset_prep(new_subset, 'thigh_gyr')
Thigh_mag_subset = dataset_prep(new_subset, 'thigh_mag')
filterscolls1 = [col for col in new_subset.columns if ('thigh_acc' in col 
                                                   or 'thigh_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('thigh_acc' in col 
                                                   or 'thigh_mag' in col 
                                                   or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('thigh_mag' in col 
                                                   or 'thigh_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls4 = [col for col in new_subset.columns if ('thigh_mag' in col
                                                      or 'thigh_gyr' in col
                                                      or 'thigh_acc' in col
                                                      or 'Activity' in col)]
Thigh_accgyr_subset = new_subset[filterscolls1] 
Thigh_accmag_subset = new_subset[filterscolls2]
Thigh_maggyr_subset = new_subset[filterscolls3]
Thigh_accmaggyr_subset = new_subset[filterscolls4] 

In [49]:
# run all models for single sensor
Thigh_acc, Thigh_acc_X = multivar_log_new(best_grid, Thigh_acc_subset)
Thigh_gyr, Thigh_gyr_X = multivar_log_new(best_grid, Thigh_gyr_subset)
Thigh_mag, Thigh_mag_X = multivar_log_new(best_grid, Thigh_mag_subset)

(4847731, 2)
(4847731,)
(1211933, 2)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.81 degrees.
[[  7164  51279  27958    161  25994     67   1895   5998]
 [  5157  84803  16835   3749  10753    179  10932   5438]
 [    88   1266  13093      3   6106      1    115     45]
 [    33   1683     60 133720     15      0  48524     26]
 [  3010  17300  62175   2280  83221      0  26795   3025]
 [    48   2872     41 107291     15      0  73375     20]
 [    60   5632     45  41273     53      0 134758     49]
 [  7788  92088  39346     83  32597    173   4140   9240]]
              precision    recall  f1-score   support

           0       0.31      0.06      0.10    120516
           1       0.33      0.62      0.43    137846
           2       0.08      0.63      0.15     20717
           3       0.46      0.73      0.57    184061
           4       0.52      0.42      0.47    197806
           5       0.00      0.00      0.00    183662
           6       0.45      0.74      0.56    181870
           7       0.39      0.05      0.09    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.74 degrees.
[[ 14215  52802  22236     75  19110    223   2619   9236]
 [ 12794  80760  11477   5800   8512    728   8028   9747]
 [  3037   8070   3826      3   4286     17    398   1080]
 [    68   1101     51  89809     25  47020  45967     20]
 [  6197  19356  29030   1741 100750  16611  11441  12680]
 [    58    959     59  53036      4  91779  37761      6]
 [   450   4887    313  22708     47  62900  90544     21]
 [ 20442  85843  30800     43  28319    184   1999  17825]]
              precision    recall  f1-score   support

           0       0.25      0.12      0.16    120516
           1       0.32      0.59      0.41    137846
           2       0.04      0.18      0.06     20717
           3       0.52      0.49      0.50    184061
           4       0.63      0.51      0.56    197806
           5       0.42      0.50      0.46    183662
           6       0.46      0.50      0.48    181870
           7       0.35      0.10      0.15    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.98 degrees.
[[ 16619  11070  32584    407  15960   4275   4929  34672]
 [ 11942  43301  29697    513  14698   8516   3001  26178]
 [  2221      0   6382      8   2107   1692   1154   7153]
 [   134   1311   6521  21843    204 133710  18110   2228]
 [ 46075     13  31385     28  22428  28132   6945  62800]
 [    82    246   6245  17746     32 131259  23126   4926]
 [   188      0  10863    634    173 142722  16720  10570]
 [ 25760  15649  30811    634  19788   8474   4557  79782]]
              precision    recall  f1-score   support

           0       0.16      0.14      0.15    120516
           1       0.60      0.31      0.41    137846
           2       0.04      0.31      0.07     20717
           3       0.52      0.12      0.19    184061
           4       0.30      0.11      0.16    197806
           5       0.29      0.71      0.41    183662
           6       0.21      0.09      0.13    181870
           7       0.35      0.43      0.39    185455

    

In [50]:
#run all models for combination sensor
Thigh_accgyr, Thigh_accgyr_X = multivar_log_new(best_grid, Thigh_accgyr_subset)
Thigh_accmag, Thigh_accmag_X = multivar_log_new(best_grid, Thigh_accmag_subset)
Thigh_maggyr, Thigh_maggyr_X = multivar_log_new(best_grid, Thigh_maggyr_subset)
Thigh_accmaggyr, Thigh_accmaggyr_X = multivar_log_new(best_grid, Thigh_accmaggyr_subset)

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.84 degrees.
[[ 19748  43912  26527    238  20316      0    647   9128]
 [ 13209  84639  15481   6109   6430      1   2967   9010]
 [   466   2314  12981     10   4306      0     49    591]
 [    70   1539     35 154223     15    154  27999     26]
 [  6167  11646  56940  14216  86941     82  14494   7320]
 [    78   1869     33 134895      6   1782  44989     10]
 [   325   5706     53  78423     33     62  97259      9]
 [ 25908  78291  38871    169  26811      3    624  14778]]
              precision    recall  f1-score   support

           0       0.30      0.16      0.21    120516
           1       0.37      0.61      0.46    137846
           2       0.09      0.63      0.15     20717
           3       0.40      0.84      0.54    184061
           4       0.60      0.44      0.51    197806
           5       0.86      0.01      0.02    183662
           6       0.51      0.53      0.52    181870
           7       0.36      0.08      0.13    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.07 degrees.
[[  4536  45436  12568   1820  29459   2620    200  23877]
 [  3474  79020   6467  10061  13111   8314    458  16941]
 [  1252   3926   6937     30   7227    619      0    726]
 [     7    945     28 146141     38  25347  11449    106]
 [ 10016  14810  29826  26043  90631   7057      8  19415]
 [    14   1453     32 145779     34  36143    111     96]
 [    31   2237     21 165963     84  13363      6    165]
 [  3631  84342  16119   3767  38628   5733    603  32632]]
              precision    recall  f1-score   support

           0       0.20      0.04      0.06    120516
           1       0.34      0.57      0.43    137846
           2       0.10      0.33      0.15     20717
           3       0.29      0.79      0.43    184061
           4       0.51      0.46      0.48    197806
           5       0.36      0.20      0.26    183662
           6       0.00      0.00      0.00    181870
           7       0.35      0.18      0.23    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.98 degrees.
[[  2727  34739  24502    347  18812      0    779  38610]
 [  2021  64952  13555   9099   7818      7   1908  38486]
 [   483   4869   5792     22   4670      0    131   4750]
 [     7   1279     85 154470     35     21  27931    233]
 [  4272   6124  24386  19814  95001   1054   7528  39627]
 [     2   1549    103 141071      2  10353  30438    144]
 [     5   9037    868 116881     45     28  54706    300]
 [  3515  57487  28681    431  23751      2    309  71279]]
              precision    recall  f1-score   support

           0       0.21      0.02      0.04    120516
           1       0.36      0.47      0.41    137846
           2       0.06      0.28      0.10     20717
           3       0.35      0.84      0.49    184061
           4       0.63      0.48      0.55    197806
           5       0.90      0.06      0.11    183662
           6       0.44      0.30      0.36    181870
           7       0.37      0.38      0.38    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.96 degrees.
[[ 20910  41771  17566    485  20415    112    635  18622]
 [ 10003  84370   8802   6666   9147    292   3559  15007]
 [  2348   2025   7716     22   4758      9    471   3368]
 [    43   1235     24 161817     31  20085    765     61]
 [ 14868   8387  31757  22661 103443   4120   3093   9477]
 [    52   1476     24 144817      9  35847   1408     29]
 [   128   3704     37 155242     39  12329  10347     44]
 [ 22544  75487  27274    505  24399     27   1658  33561]]
              precision    recall  f1-score   support

           0       0.29      0.17      0.22    120516
           1       0.39      0.61      0.47    137846
           2       0.08      0.37      0.14     20717
           3       0.33      0.88      0.48    184061
           4       0.64      0.52      0.57    197806
           5       0.49      0.20      0.28    183662
           6       0.47      0.06      0.10    181870
           7       0.42      0.18      0.25    185455

    

#### Single sensor Upperarm

In [51]:
#create subsets for upperarm
upperarm_acc_subset = dataset_prep(new_subset, 'upperarm_acc')
upperarm_gyr_subset = dataset_prep(new_subset, 'upperarm_gyr')
upperarm_mag_subset = dataset_prep(new_subset, 'upperarm_mag')
filterscolls1 = [col for col in new_subset.columns if ('upperarm_acc' in col 
                                                   or 'upperarm_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('upperarm_acc' in col 
                                                   or 'upperarm_mag' in col 
                                                   or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('upperarm_mag' in col 
                                                   or 'upperarm_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls4 = [col for col in new_subset.columns if ('upperarm_mag' in col
                                                      or 'upperarm_gyr' in col
                                                      or 'upperarm_acc' in col
                                                      or 'Activity' in col)]
upperarm_accgyr_subset = new_subset[filterscolls1] 
upperarm_accmag_subset = new_subset[filterscolls2]
upperarm_maggyr_subset = new_subset[filterscolls3]
upperarm_accmaggyr_subset = new_subset[filterscolls4] 

In [52]:
#run all models for single sensor
upperarm_acc, upperarm_acc_X = multivar_log_new(best_grid, upperarm_acc_subset)
upperarm_gyr, upperarm_gyr_X = multivar_log_new(best_grid, upperarm_gyr_subset)
upperarm_mag, upperarm_mag_X = multivar_log_new(best_grid, upperarm_mag_subset)

(4847731, 2)
(4847731,)
(1211933, 2)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.77 degrees.
[[ 23278  37898  15168   2435  17444    350   2676  21267]
 [ 18839  64548   4506   3122   4600   1896  14317  26018]
 [   358   1300  13619     23   4525      0    514    378]
 [   200   2028     84 127605     36  43204  10558    346]
 [ 17171  18060  61309    541  64696    638  26127   9264]
 [   288  10751     84   4898     47  31625 135313    656]
 [   295  14059     59    135     38   1566 164989    729]
 [ 36129  75942  12396   3735   8146    737   5410  42960]]
              precision    recall  f1-score   support

           0       0.24      0.19      0.21    120516
           1       0.29      0.47      0.36    137846
           2       0.13      0.66      0.21     20717
           3       0.90      0.69      0.78    184061
           4       0.65      0.33      0.44    197806
           5       0.40      0.17      0.24    183662
           6       0.46      0.91      0.61    181870
           7       0.42      0.23      0.30    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.87 degrees.
[[ 14781  41703  26599    225  14875    219   9329  12785]
 [ 12696  64885  14248   3872  12605   1448  14925  13167]
 [  2698   4060   6594      0   4002     19   1299   2045]
 [   210   1266    226 174896    280   5088   1858    237]
 [  7629  18180  31567   9173 107817   8413  10207   4820]
 [   870   4539    819 128064    438  29949  17716   1267]
 [  1124   9641   1036  68754    595  50852  48077   1791]
 [ 23308  78074  31753    233  14746    333  15744  21264]]
              precision    recall  f1-score   support

           0       0.23      0.12      0.16    120516
           1       0.29      0.47      0.36    137846
           2       0.06      0.32      0.10     20717
           3       0.45      0.95      0.61    184061
           4       0.69      0.55      0.61    197806
           5       0.31      0.16      0.21    183662
           6       0.40      0.26      0.32    181870
           7       0.37      0.11      0.18    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.28 degrees.
[[  8430  39988     17  22801   7207   4044   1048  36981]
 [ 18668  46660     12  29899   7242   2139   2254  30972]
 [  2680   4421      1   3587   1043   2169    153   6663]
 [  6228   6475      0 144584    139  22817   1958   1860]
 [ 17121  52916      0  31167  27410   3495   1306  64391]
 [  3005   7834     38  99815    368  65472    739   6391]
 [  3722  12966    108 129689    385  28816    721   5463]
 [ 19800  61252     18  29548   8265   4255   2168  60149]]
              precision    recall  f1-score   support

           0       0.11      0.07      0.08    120516
           1       0.20      0.34      0.25    137846
           2       0.01      0.00      0.00     20717
           3       0.29      0.79      0.43    184061
           4       0.53      0.14      0.22    197806
           5       0.49      0.36      0.41    183662
           6       0.07      0.00      0.01    181870
           7       0.28      0.32      0.30    185455

    

In [53]:
#run all models for combination sensor
upperarm_accgyr, upperarm_accgyr_X = multivar_log_new(best_grid, upperarm_accgyr_subset)
upperarm_accmag, upperarm_accmag_X = multivar_log_new(best_grid, upperarm_accmag_subset)
upperarm_maggyr, upperarm_maggyr_X = multivar_log_new(best_grid, upperarm_maggyr_subset)
upperarm_accmaggyr, upperarm_accmaggyr_X = multivar_log_new(best_grid, upperarm_accmaggyr_subset)

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.57 degrees.
[[ 29006  42848  19045      2  16190    304   1957  11164]
 [ 19032  77251   4047    589  10600   4324   6763  15240]
 [   805   1706  13849      0   3241      0    513    603]
 [   201   1778     66 161708    237  17342   2591    138]
 [  9363  11000  40784    432 108365  15081  11467   1314]
 [   411   7332     75  32445    301 118547  24085    466]
 [   398  12259     58    595    429 107219  60465    447]
 [ 39424  88647  12770      3  13763    329   2551  27968]]
              precision    recall  f1-score   support

           0       0.29      0.24      0.26    120516
           1       0.32      0.56      0.41    137846
           2       0.15      0.67      0.25     20717
           3       0.83      0.88      0.85    184061
           4       0.71      0.55      0.62    197806
           5       0.45      0.65      0.53    183662
           6       0.55      0.33      0.41    181870
           7       0.49      0.15      0.23    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.98 degrees.
[[ 19881  33322   9383   8761  40992   3527      0   4650]
 [ 23905  55508   1863  28211  21097   5285      0   1977]
 [   534   2591  12488     14   2935   1249      0    906]
 [   295   1189     38 176129    222   6170      0     18]
 [ 12642  13875  59232  24831  72635   7520      0   7071]
 [   536   3973     45 113184    310  65498      0    116]
 [   663   4919     23 143404    286  32496      0     79]
 [ 41193  60800   6180  15891  45433   6946      0   9012]]


...

              precision    recall  f1-score   support

           0       0.20      0.16      0.18    120516
           1       0.32      0.40      0.35    137846
           2       0.14      0.60      0.23     20717
           3       0.35      0.96      0.51    184061
           4       0.39      0.37      0.38    197806
           5       0.51      0.36      0.42    183662
           6       0.00      0.00      0.00    181870
           7       0.38      0.05      0.09    185455

    accuracy                           0.34   1211933
   macro avg       0.29      0.36      0.27   1211933
weighted avg       0.31      0.34      0.28   1211933

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.88 degrees.
[[ 14511  50931  23455    420  19975    181   4355   6688]
 [  7278  78649  14729   5695  16155     87   7726   7527]
 [  1301   5946   6901     11   4658     57    591   1252]
 [   124   1600    233 156405    326  23832   1404    137]
 [ 11517  18682  26665  15633 112296   3473   6518   3022]
 [   515   7774    845  94432    508  66097  12470   1021]
 [   357  16442   1196  99010    758  27736  35147   1224]
 [ 23865  87532  31861    398  20522    167   6599  14511]]
              precision    recall  f1-score   support

           0       0.24      0.12      0.16    120516
           1       0.29      0.57      0.39    137846
           2       0.07      0.33      0.11     20717
           3       0.42      0.85      0.56    184061
           4       0.64      0.57      0.60    197806
           5       0.54      0.36      0.43    183662
           6       0.47      0.19      0.27    181870
           7       0.41      0.08      0.13    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.92 degrees.
[[ 30509  39878  19232   2775  15377     15   1480  11250]
 [ 26577  76748   4998  10671   6596     11   4979   7266]
 [  2846   1455  10830     12   3159      3    805   1607]
 [   266   1652     57 181525    179      9    310     63]
 [ 13652  10811  39579  21565 100007   2438   4033   5721]
 [   432   6097     86 118302    206  23977  34350    212]
 [   440  10269     59 133328    271  12349  25005    149]
 [ 50007  79893  13623   1887  11132     72   4817  24024]]
              precision    recall  f1-score   support

           0       0.24      0.25      0.25    120516
           1       0.34      0.56      0.42    137846
           2       0.12      0.52      0.20     20717
           3       0.39      0.99      0.56    184061
           4       0.73      0.51      0.60    197806
           5       0.62      0.13      0.22    183662
           6       0.33      0.14      0.19    181870
           7       0.48      0.13      0.20    185455

    

#### Single sensor Chest

In [54]:
#create subsets for Chest
Chest_acc_subset = dataset_prep(new_subset, 'chest_acc')
Chest_gyr_subset = dataset_prep(new_subset, 'chest_gyr')
Chest_mag_subset = dataset_prep(new_subset, 'chest_mag')
filterscolls1 = [col for col in new_subset.columns if ('chest_acc' in col 
                                                   or 'chest_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('chest_acc' in col 
                                                   or 'chest_mag' in col 
                                                   or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('chest_mag' in col 
                                                   or 'chest_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls4 = [col for col in new_subset.columns if ('chest_mag' in col
                                                      or 'chest_gyr' in col
                                                      or 'chest_acc' in col
                                                      or 'Activity' in col)]
Chest_accgyr_subset = new_subset[filterscolls1] 
Chest_accmag_subset = new_subset[filterscolls2]
Chest_maggyr_subset = new_subset[filterscolls3]
Chest_accmaggyr_subset = new_subset[filterscolls4] 

In [55]:
#run all models for single sensor
Chest_acc, Chest_acc_X = multivar_log_new(best_grid, Chest_acc_subset)
Chest_gyr, Chest_gyr_X = multivar_log_new(best_grid, Chest_gyr_subset)
Chest_mag, Chest_mag_X = multivar_log_new(best_grid, Chest_mag_subset)

(4847731, 2)
(4847731,)
(1211933, 2)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.98 degrees.
[[ 30355  46358  11158   1036  16792    449   2540  11828]
 [ 24165  73928   3173   1595   5375   6685   7927  14998]
 [  3775   1678  10556      0   3480     53    491    684]
 [   111   2420     24     15     17 162520  18848    106]
 [ 30188  17745  75698     38  42732  13945  11380   6080]
 [    74   5822      8      8      9 152001  25669     71]
 [    91  11186     10      9      9 128996  41448    121]
 [ 49155  83647   9851    682  11525    702   4636  25257]]
              precision    recall  f1-score   support

           0       0.22      0.25      0.23    120516
           1       0.30      0.54      0.39    137846
           2       0.10      0.51      0.16     20717
           3       0.00      0.00      0.00    184061
           4       0.53      0.22      0.31    197806
           5       0.33      0.83      0.47    183662
           6       0.37      0.23      0.28    181870
           7       0.43      0.14      0.21    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.68 degrees.
[[ 11539  33290  22754    317   5901    937   6717  39061]
 [  8034  81211   6218    343   3637   1195  10521  26687]
 [  1334   2345   7964     16   3848    148    840   4222]
 [   120    933    131 141976    281   9831  30321    468]
 [  3624  11447  33580  10852 121153   5613   8682   2855]
 [   145   2642    255 102589    186  34387  41290   2168]
 [   109  10748    185  62163     88  36333  67953   4291]
 [ 12884  48275  18363    586   4266   1791  18123  81167]]
              precision    recall  f1-score   support

           0       0.31      0.10      0.15    120516
           1       0.43      0.59      0.49    137846
           2       0.09      0.38      0.14     20717
           3       0.45      0.77      0.56    184061
           4       0.87      0.61      0.72    197806
           5       0.38      0.19      0.25    183662
           6       0.37      0.37      0.37    181870
           7       0.50      0.44      0.47    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.0 degrees.
[[    0 31929  6784    82 21687 12568 13509 33957]
 [    0 42365 12159    50 25329 15731 12471 29741]
 [    0  3312  4380     2  5407  1137  2473  4006]
 [    0  8191  1322   325   778 97067 67344  9034]
 [    0 30685 36451    36 81839  9424 12467 26904]
 [    0  9656   205   133  1548 81117 76498 14505]
 [    0 15250   454   278  3085 66605 68204 27994]
 [    0 39787  5443   282 19440 28602 32588 59313]]


C:\Users\31657\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\31657\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    120516
           1       0.23      0.31      0.27    137846
           2       0.07      0.21      0.10     20717
           3       0.27      0.00      0.00    184061
           4       0.51      0.41      0.46    197806
           5       0.26      0.44      0.33    183662
           6       0.24      0.38      0.29    181870
           7       0.29      0.32      0.30    185455

    accuracy                           0.28   1211933
   macro avg       0.23      0.26      0.22   1211933
weighted avg       0.27      0.28      0.25   1211933



C:\Users\31657\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
### run all models for combination sensor
Chest_accgyr, Chest_accgyr_X = multivar_log_new(best_grid, Chest_accgyr_subset)
Chest_accmag, Chest_accmag_X = multivar_log_new(best_grid, Chest_accmag_subset)
Chest_maggyr, Chest_maggyr_X = multivar_log_new(best_grid, Chest_maggyr_subset)
Chest_accmaggyr, Chest_accmaggyr_X = multivar_log_new(best_grid, Chest_accmaggyr_subset)

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.63 degrees.
[[ 19829  42176  19005    210   6463     51   2117  30665]
 [ 11723  74173   3325    167   3750     53   7580  37075]
 [  1435   2231  11343      0   3735      1    570   1402]
 [    28   1139      9 141222    223  24378  16340    722]
 [  1494  11740  33463   8754 124149   2297  10736   5173]
 [    27   1632     14 101801    121  34121  43427   2519]
 [    46   2946     11  58563     91  39379  75244   5590]
 [ 28771  50033  12128    327   4883     80   4785  84448]]
              precision    recall  f1-score   support

           0       0.31      0.16      0.22    120516
           1       0.40      0.54      0.46    137846
           2       0.14      0.55      0.23     20717
           3       0.45      0.77      0.57    184061
           4       0.87      0.63      0.73    197806
           5       0.34      0.19      0.24    183662
           6       0.47      0.41      0.44    181870
           7       0.50      0.46      0.48    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.14 degrees.
[[ 40587  15797  17071   3755  11114   2738   3203  26251]
 [ 34051  29871   4317  12305   4684   7013  12062  33543]
 [  3389    703   7237   1850   6939      4      5    590]
 [   178   1101     21  18565     17  55486 108378    315]
 [ 30251   7570  66382   8848  54047  11714  12125   6869]
 [   102   1463      9  42826      9  52237  86641    375]
 [   156   2647     20  31955      5  34027 112436    624]
 [ 62603  31559  10247   9796  12329   4352   7513  47056]]
              precision    recall  f1-score   support

           0       0.24      0.34      0.28    120516
           1       0.33      0.22      0.26    137846
           2       0.07      0.35      0.11     20717
           3       0.14      0.10      0.12    184061
           4       0.61      0.27      0.38    197806
           5       0.31      0.28      0.30    183662
           6       0.33      0.62      0.43    181870
           7       0.41      0.25      0.31    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.78 degrees.
[[ 42194  11625  13635    677   8663    220   5605  37897]
 [ 33135  44595   3739    967   8756    227   6923  39504]
 [  4670   1194   7699     80   2381     83    703   3907]
 [   400    556    477 157854    428   5608  18061    677]
 [  8395   4233  57356  15651  98835   1868   6583   4885]
 [   744    723    147 111286    256  33747  27038   9721]
 [  1387   2162     87  85639    203  17641  54418  20333]
 [ 57161  10800   7415   1174   6067    464  14570  87804]]
              precision    recall  f1-score   support

           0       0.28      0.35      0.31    120516
           1       0.59      0.32      0.42    137846
           2       0.09      0.37      0.14     20717
           3       0.42      0.86      0.57    184061
           4       0.79      0.50      0.61    197806
           5       0.56      0.18      0.28    183662
           6       0.41      0.30      0.34    181870
           7       0.43      0.47      0.45    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.84 degrees.
[[ 52166  23382  18939   1806   6891   1731   2105  13496]
 [ 45223  48008   4941   5894   3425   3332  10105  16918]
 [  3539    707   9829   1733   4290     10      6    603]
 [   192   1383      8  27668    123 153690    805    192]
 [ 10332  13139  27357   7103 118229  16357   2795   2494]
 [   142   1654      4  58700     53 111229  11432    448]
 [   219   2953      8  43879     18 112533  21297    963]
 [ 76230  36769  14800   5589   4223   4627   6725  36492]]
              precision    recall  f1-score   support

           0       0.28      0.43      0.34    120516
           1       0.38      0.35      0.36    137846
           2       0.13      0.47      0.20     20717
           3       0.18      0.15      0.16    184061
           4       0.86      0.60      0.71    197806
           5       0.28      0.61      0.38    183662
           6       0.39      0.12      0.18    181870
           7       0.51      0.20      0.28    185455

    

#### Single sensor Shin

In [57]:
#create subsets for Shin
Shin_acc_subset = dataset_prep(new_subset, 'shin_acc')
Shin_gyr_subset = dataset_prep(new_subset, 'shin_gyr')
Shin_mag_subset = dataset_prep(new_subset, 'shin_mag')
filterscolls1 = [col for col in new_subset.columns if ('shin_acc' in col 
                                                   or 'shin_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('shin_acc' in col 
                                                   or 'shin_mag' in col 
                                                   or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('shin_mag' in col 
                                                   or 'shin_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls4 = [col for col in new_subset.columns if ('shin_mag' in col
                                                      or 'shin_gyr' in col
                                                      or 'shin_acc' in col
                                                      or 'Activity' in col)]
Shin_accgyr_subset = new_subset[filterscolls1] 
Shin_accmag_subset = new_subset[filterscolls2]
Shin_maggyr_subset = new_subset[filterscolls3]
Shin_accmaggyr_subset = new_subset[filterscolls4] 

In [58]:
#run all models for single sensor
Shin_acc, Shin_acc_X = multivar_log_new(best_grid, Shin_acc_subset)
Shin_gyr, Shin_gyr_X = multivar_log_new(best_grid, Shin_gyr_subset)
Shin_mag, Shin_mag_X = multivar_log_new(best_grid, Shin_mag_subset)

(4847731, 2)
(4847731,)
(1211933, 2)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.94 degrees.
[[  5772  40960  35266      9  29656   4503    380   3970]
 [  3652  57336  22655     98  19471  20848   7507   6279]
 [   322   2990   9960      3   6826    113      1    502]
 [    59   2067    131     15     25  10394 171351     19]
 [  3030  16285  46360      2  96591  10095  20802   4641]
 [   333   7854    541      4     14  28569 146319     28]
 [    81   2852    128      2     21  31593 147167     26]
 [  4824  67105  38753      5  52533  15092    468   6675]]
              precision    recall  f1-score   support

           0       0.32      0.05      0.08    120516
           1       0.29      0.42      0.34    137846
           2       0.06      0.48      0.11     20717
           3       0.11      0.00      0.00    184061
           4       0.47      0.49      0.48    197806
           5       0.24      0.16      0.19    183662
           6       0.30      0.81      0.44    181870
           7       0.30      0.04      0.06    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.93 degrees.
[[ 29078  38340  21681    316  26675     17   1623   2786]
 [ 26431  60966  23599   7025   7197    303  10068   2257]
 [  2558   5361   9959      1   2246      0     42    550]
 [   109   1928    149 177115     29    709   4018      4]
 [ 11113  14566  30717  22952 108773   2100   5034   2551]
 [   128   7788    347 159063     24   3163  13143      6]
 [   157   3589    335 130904     34  15841  30999     11]
 [ 36319  65316  29715    369  50287     17    436   2996]]
              precision    recall  f1-score   support

           0       0.27      0.24      0.26    120516
           1       0.31      0.44      0.36    137846
           2       0.09      0.48      0.15     20717
           3       0.36      0.96      0.52    184061
           4       0.56      0.55      0.55    197806
           5       0.14      0.02      0.03    183662
           6       0.47      0.17      0.25    181870
           7       0.27      0.02      0.03    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.14 degrees.
[[ 21979  30321  23665     56  22410   2858   2738  16489]
 [ 14074  34421  32692   5405  19626   8046   8481  15101]
 [  2670   7252   4251      2   2425    978    914   2225]
 [   118   7025   7021  60674    288  54403  54426    106]
 [ 27247  57909  18547   4497  38645  22985   3926  24050]
 [   372   8783   4173  27683    479  99940  40287   1945]
 [   183  13604   6254  21381    184 100740  39305    219]
 [ 33215  56821  28579     33  34640   4133   2716  25318]]
              precision    recall  f1-score   support

           0       0.22      0.18      0.20    120516
           1       0.16      0.25      0.19    137846
           2       0.03      0.21      0.06     20717
           3       0.51      0.33      0.40    184061
           4       0.33      0.20      0.24    197806
           5       0.34      0.54      0.42    183662
           6       0.26      0.22      0.23    181870
           7       0.30      0.14      0.19    185455

    

In [59]:
#run all models for combination sensor
Shin_accgyr, Shin_accgyr_X = multivar_log_new(best_grid, Shin_accgyr_subset)
Shin_accmag, Shin_accmag_X = multivar_log_new(best_grid, Shin_accmag_subset)
Shin_maggyr, Shin_maggyr_X = multivar_log_new(best_grid, Shin_maggyr_subset)
Shin_accmaggyr, Shin_accmaggyr_X = multivar_log_new(best_grid, Shin_accmaggyr_subset)

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.93 degrees.
[[ 19499  36293  23945      0  23270   1064    385  16060]
 [ 10232  56737  23094      0   9527   9668   7321  21267]
 [  1833   3431  11352      0   3210     29      1    861]
 [    28   2042    137      0     36  17800 163891    127]
 [ 11188  19574  20443      0 108397  13577  16599   8028]
 [    22   8588    683      1     26  41147 132972    223]
 [    36   3089    198      0     42  42692 135577    236]
 [ 18008  65946  23598      0  47193   1165    421  29124]]
              precision    recall  f1-score   support

           0       0.32      0.16      0.22    120516
           1       0.29      0.41      0.34    137846
           2       0.11      0.55      0.18     20717
           3       0.00      0.00      0.00    184061
           4       0.57      0.55      0.56    197806
           5       0.32      0.22      0.26    183662
           6       0.30      0.75      0.42    181870
           7       0.38      0.16      0.22    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.13 degrees.
[[ 16559  28900  30627   1475  24796      0   7272  10887]
 [ 10719  39022  20052   5193  17531      0  34294  11035]
 [  1912   1729   9729     12   6005      0    832    498]
 [   125   1463     90  57955     40      0 124257    131]
 [ 12014   6715  46261  18383  82212      0  25719   6502]
 [   490   5242    446  60757    108      0 116093    526]
 [   120   1845    109  84276     31      0  95303    186]
 [ 18853  48275  32888   5771  45136      0  20319  14213]]


...

              precision    recall  f1-score   support

           0       0.27      0.14      0.18    120516
           1       0.29      0.28      0.29    137846
           2       0.07      0.47      0.12     20717
           3       0.25      0.31      0.28    184061
           4       0.47      0.42      0.44    197806
           5       0.00      0.00      0.00    183662
           6       0.22      0.52      0.31    181870
           7       0.32      0.08      0.12    185455

    accuracy                           0.26   1211933
   macro avg       0.24      0.28      0.22   1211933
weighted avg       0.26      0.26      0.23   1211933

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.77 degrees.
[[ 31417  38713  18916     56  24281   1416    267   5450]
 [ 26797  63884  19085     18   7654  14321   1619   4468]
 [  2503   6239   8109      0   2822     32      2   1010]
 [   188   2054     94  26592     28 153129   1962     14]
 [ 12958  16608  27030      7 107832  29748    225   3398]
 [   412   8159    207  13986     29 139248  21608     13]
 [   234   4332    301   5290     47 156533  15100     33]
 [ 41336  64493  27692      1  44214    726     31   6962]]
              precision    recall  f1-score   support

           0       0.27      0.26      0.27    120516
           1       0.31      0.46      0.37    137846
           2       0.08      0.39      0.13     20717
           3       0.58      0.14      0.23    184061
           4       0.58      0.55      0.56    197806
           5       0.28      0.76      0.41    183662
           6       0.37      0.08      0.14    181870
           7       0.33      0.04      0.07    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.01 degrees.
[[ 17651  31983  22594   1324  24555    957   1444  20008]
 [ 11810  43053  21684   6044  10359   9680  11758  23458]
 [  3211   2882  10376     24   3307      6    243    668]
 [    50   1350    168  93148     29  30880  58181    255]
 [  8908  17084  17205  20955 106790   1718  12082  13064]
 [    31   4818   1201  86916      6  39522  50566    602]
 [    43   1832    221 105918     17  20266  53202    371]
 [ 15353  56837  19350   2541  52530   1900   3440  33504]]
              precision    recall  f1-score   support

           0       0.31      0.15      0.20    120516
           1       0.27      0.31      0.29    137846
           2       0.11      0.50      0.18     20717
           3       0.29      0.51      0.37    184061
           4       0.54      0.54      0.54    197806
           5       0.38      0.22      0.27    183662
           6       0.28      0.29      0.29    181870
           7       0.36      0.18      0.24    185455

    

## Combination (two sensor positions) - Subq 2
Logistic combination of sensor-positions
(head, waist, shin (scheenbeen), upperarm, forearm (onderarm), chest, thigh (pocket)) <br>

Head = smartglasses <br>
forearm = smartwatch <br>
thigh = pocket smartwatch <br>

Combinations: <br>
Head - Forearm <br>
Head - Thigh <br>
Forearm - Thigh <br>

Different Sensors: <br>
acc (mean, sd) <br>
gyr (mean, sd) <br>
mag (mean, sd) <br>
acc + mag (mean, sd) <br>
acc + gyr (mean, sd) <br>
gyr + mag (mean, sd) <br>
acc + gyr + mag (mean, sd) <br>

### Each combo - only acc

In [60]:
#create subsets for first combinations with only acc
filterscolls1 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                   or 'head_acc' in col 
                                                   or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('thigh_acc' in col 
                                                   or 'head_acc' in col 
                                                   or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                   or 'thigh_acc' in col 
                                                   or 'Activity' in col)]
headforearm_acc_subset = new_subset[filterscolls1] 
headthigh_acc_subset = new_subset[filterscolls2]  
thighforearm_acc_subset = new_subset[filterscolls3] 

In [61]:
#run all models for combination sensor
headforearm_acc, headforearm_acc_X = multivar_log_new(best_grid, headforearm_acc_subset)
headthigh_acc, headthigh_acc_X = multivar_log_new(best_grid, headthigh_acc_subset)
thighforearm_acc, thighforearm_acc_X = multivar_log_new(best_grid, thighforearm_acc_subset)

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.98 degrees.
[[ 31594  33813  11409    280  12673    499   1797  28451]
 [ 19124  70710   2267   4757   1621   2837   6525  30005]
 [  1417   1457  11501      2   4937     66    280   1057]
 [   252   2893     71 169364     18   2928   8206    329]
 [ 12585   8726  54439  11483  84251   3594  10653  12075]
 [   542  11299    245 127175      6  15837  28422    136]
 [   499  13027    181 103590     22  17881  46373    297]
 [ 36119  76008   5841    307   8970   1657   3508  53045]]
              precision    recall  f1-score   support

           0       0.31      0.26      0.28    120516
           1       0.32      0.51      0.40    137846
           2       0.13      0.56      0.22     20717
           3       0.41      0.92      0.56    184061
           4       0.75      0.43      0.54    197806
           5       0.35      0.09      0.14    183662
           6       0.44      0.25      0.32    181870
           7       0.42      0.29      0.34    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.88 degrees.
[[ 26286  35920  11236    223  13549    312    957  32033]
 [ 19736  75759   2798   5830   2728   2178   4606  24211]
 [   870   1878  11668      1   4968      2     95   1235]
 [   229   1489      5 157029     20  22299   2904     86]
 [ 12574   6097  52074  13299  88115   8526   7215   9906]
 [    54   2669      2  86002      1  83638  11267     29]
 [   128   4830      0 109500      4  40136  27210     62]
 [ 38391  70339  12081    349  16376    401   2182  45336]]
              precision    recall  f1-score   support

           0       0.27      0.22      0.24    120516
           1       0.38      0.55      0.45    137846
           2       0.13      0.56      0.21     20717
           3       0.42      0.85      0.56    184061
           4       0.70      0.45      0.54    197806
           5       0.53      0.46      0.49    183662
           6       0.48      0.15      0.23    181870
           7       0.40      0.24      0.30    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.85 degrees.
[[ 21931  31548  13468    272  21730    293    992  30282]
 [ 17584  70690   4067   3823   7018   2060   7417  25187]
 [   938   1151  11976     28   4513     65     50   1996]
 [   393   1644     32 145022     52   6761  30055    102]
 [  7909   9962  55546   4394  78068   1771  21598  18558]
 [  1250   4090    136  77454    102  43722  56818     90]
 [   996   5338     72  62905    136   5295 106977    151]
 [ 26892  73623  11454    170  20888    841   2241  49346]]
              precision    recall  f1-score   support

           0       0.28      0.18      0.22    120516
           1       0.36      0.51      0.42    137846
           2       0.12      0.58      0.20     20717
           3       0.49      0.79      0.61    184061
           4       0.59      0.39      0.47    197806
           5       0.72      0.24      0.36    183662
           6       0.47      0.59      0.52    181870
           7       0.39      0.27      0.32    185455

    

### Each combo - only gyr

In [62]:
#create subsets for first combinations with only gyr
filterscolls1 = [col for col in new_subset.columns if ('thigh_gyr' in col 
                                                   or 'head_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('forearm_gyr' in col 
                                                   or 'thigh_gyr' in col 
                                                   or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('forearm_gyr' in col 
                                                   or 'head_gyr' in col 
                                                   or 'Activity' in col)]
 
headthigh_gyr_subset = new_subset[filterscolls1]  
thighforearm_gyr_subset = new_subset[filterscolls2] 
headforearm_gyr_subset = new_subset[filterscolls3] 

In [63]:
#run all models for combination sensor
headthigh_gyr, headthigh_gyr_X = multivar_log_new(best_grid, headthigh_gyr_subset)
thighforearm_gyr, thighforearm_gyr_X = multivar_log_new(best_grid, thighforearm_gyr_subset)
headforearm__gyr, headforearm__gyr_X = multivar_log_new(best_grid, headforearm_gyr_subset)

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.76 degrees.
[[ 15421  36970  20497    211  11779    132   2422  33084]
 [ 10821  70112   7902   4069   3820   1887   7631  31604]
 [  1417   2357  10574      0   3329     19    361   2660]
 [    55   1057    112 148127     43  30532   4022    113]
 [ 14413  10088  43784  11584  83196   7253  10523  16965]
 [    55   1088    139 114776      2  35402  32070    130]
 [   177   5077    396  68960     54  37628  68658    920]
 [ 16802  58763  14399    207  11880    122   1841  81441]]
              precision    recall  f1-score   support

           0       0.26      0.13      0.17    120516
           1       0.38      0.51      0.43    137846
           2       0.11      0.51      0.18     20717
           3       0.43      0.80      0.56    184061
           4       0.73      0.42      0.53    197806
           5       0.31      0.19      0.24    183662
           6       0.54      0.38      0.44    181870
           7       0.49      0.44      0.46    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.77 degrees.
[[ 15280  36039  32359    280  18818     84   1525  16131]
 [ 11027  69318  21298   3658   5397   1919   6070  19159]
 [  1794   4540   9364      0   3240      1    240   1538]
 [    72   1057    159 168869     14  10862   2965     63]
 [ 11344   7252  31182   8371 107189  10050  10812  11606]
 [   285    905    388 123023     58  43606  15374     23]
 [   648   4789    939  73284    139  48597  53298    176]
 [ 17186  70444  40989    170  18442    103   1104  37017]]
              precision    recall  f1-score   support

           0       0.27      0.13      0.17    120516
           1       0.36      0.50      0.42    137846
           2       0.07      0.45      0.12     20717
           3       0.45      0.92      0.60    184061
           4       0.70      0.54      0.61    197806
           5       0.38      0.24      0.29    183662
           6       0.58      0.29      0.39    181870
           7       0.43      0.20      0.27    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.81 degrees.
[[ 16950  27275  20800    201  18447    297   8566  27980]
 [ 13959  52660  10068   2689   7095   1580  14898  34897]
 [  1078   1169  11811      0   3161     78   1357   2063]
 [   213    922    316 167536    167   9395   4222   1290]
 [ 22876  12453  42148   8145  81953   7453  10428  12350]
 [  1084   3586   2679 113939    699  31982  24455   5238]
 [  1541   5918   3266  59080    969  50395  53159   7542]
 [ 19160  43108  18621    157   8666    479  17345  77919]]
              precision    recall  f1-score   support

           0       0.22      0.14      0.17    120516
           1       0.36      0.38      0.37    137846
           2       0.11      0.57      0.18     20717
           3       0.48      0.91      0.63    184061
           4       0.68      0.41      0.51    197806
           5       0.31      0.17      0.22    183662
           6       0.40      0.29      0.34    181870
           7       0.46      0.42      0.44    185455

    

### Each combo - only mag

In [64]:
#create subsets for first combinations with only mag
filterscolls2 = [col for col in new_subset.columns if ('thigh_mag' in col 
                                                   or 'head_mag' in col 
                                                   or 'Activity' in col)]
filterscolls4 = [col for col in new_subset.columns if ('forearm_mag' in col 
                                                   or 'thigh_mag' in col 
                                                   or 'Activity' in col)]
filterscolls6 = [col for col in new_subset.columns if ('forearm_mag' in col 
                                                   or 'head_mag' in col 
                                                   or 'Activity' in col)]

headthigh_mag_subset = new_subset[filterscolls2] 
thighforearm_mag_subset = new_subset[filterscolls4] 
headforearm_mag_subset = new_subset[filterscolls6] 

In [65]:
#run all models for combination sensor
headthigh_mag, headthigh_mag_X = multivar_log_new(best_grid, headthigh_mag_subset)
thighforearm_mag, thighforearm_mag_X = multivar_log_new(best_grid, thighforearm_mag_subset)
headforearm__mag, headforearm__mag_X = multivar_log_new(best_grid, headforearm_mag_subset)

(4847731, 4)
(4847731,)
(1211933, 4)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.01 degrees.
[[23787 11595 24165  2964 25124  1460  3357 28064]
 [10828 41839 23119  8824 17972  1494  2623 31147]
 [ 4789    89  4653  1738  3925   239   881  4403]
 [  510    96  5842 90185   887 52314 34014   213]
 [27778   655 28577  9594 43735  3266 18272 65929]
 [ 3765   103  7110 43470  1767 71765 55497   185]
 [ 3568     1 17113 47322  3470 42402 67589   405]
 [22120 20947 25323  3678 49115  1332  8103 54837]]
              precision    recall  f1-score   support

           0       0.24      0.20      0.22    120516
           1       0.56      0.30      0.39    137846
           2       0.03      0.22      0.06     20717
           3       0.43      0.49      0.46    184061
           4       0.30      0.22      0.25    197806
           5       0.41      0.39      0.40    183662
           6       0.36      0.37      0.36    181870
           7       0.30      0.30      0.30    185455

    accuracy                           0.33   1211933
   macro avg  

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.03 degrees.
[[11696 13029 14170  3799 27278  7633  6809 36102]
 [10599 41039 10083  8642 21800  8497  4694 32492]
 [ 1842  1139  2821   696  5076  2977   957  5209]
 [  220   152  9408 87898 15468 44825 25578   512]
 [14940 12268 17578 10725 54548 14867 14143 58737]
 [  637    26  8575 49922 14784 91187 18109   422]
 [  600    38 14134 58944 16290 61869 28985  1010]
 [17533 23619 15962  6119 32150 11384 11379 67309]]
              precision    recall  f1-score   support

           0       0.20      0.10      0.13    120516
           1       0.45      0.30      0.36    137846
           2       0.03      0.14      0.05     20717
           3       0.39      0.48      0.43    184061
           4       0.29      0.28      0.28    197806
           5       0.37      0.50      0.43    183662
           6       0.26      0.16      0.20    181870
           7       0.33      0.36      0.35    185455

    accuracy                           0.32   1211933
   macro avg  

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.09 degrees.
[[11262 20656  1811 19065 28793 17715  8929 12285]
 [ 8654 21611  2750 30082 32403 18600  8731 15015]
 [ 1127  2946  1614  3345  4948  2774  1852  2111]
 [ 2846  7056  1943 72296 36919 23683 35062  4256]
 [13823 29572  2174 25846 61511 26685 16339 21856]
 [11348  8378  5050 10455 38461 79671 19785 10514]
 [ 4898 21882  2966 39547 40742 35826 24685 11324]
 [12546 26813  1252 21511 40630 35663 21433 25607]]
              precision    recall  f1-score   support

           0       0.17      0.09      0.12    120516
           1       0.16      0.16      0.16    137846
           2       0.08      0.08      0.08     20717
           3       0.33      0.39      0.36    184061
           4       0.22      0.31      0.26    197806
           5       0.33      0.43      0.38    183662
           6       0.18      0.14      0.15    181870
           7       0.25      0.14      0.18    185455

    accuracy                           0.25   1211933
   macro avg  

### Each combo - only acc + gyr

In [12]:
#create subsets for first combinations with only acc+gyr
filterscolls1 = [col for col in new_subset.columns if ('thigh_acc' in col 
                                                       or 'thigh_gyr' in col
                                                       or 'head_acc' in col
                                                       or 'head_gyr' in col
                                                       or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                       or 'thigh_gyr' in col
                                                       or 'thigh_acc' in col
                                                       or 'forearm_gyr' in col 
                                                       or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                       or 'head_acc' in col
                                                       or 'head_gyr' in col
                                                       or 'forearm_gyr' in col
                                                       or 'Activity' in col)]
 
headthigh_accgyr_subset = new_subset[filterscolls1] 
thighforearm_accgyr_subset = new_subset[filterscolls2] 
headforearm_accgyr_subset = new_subset[filterscolls3] 

In [13]:
#run all models for combination sensor
headthigh_accgyr, headthigh_accgyr_X = multivar_log_new(best_grid, headthigh_accgyr_subset)
thighforearm_accgyr, thighforearm_accgyr_X = multivar_log_new(best_grid, thighforearm_accgyr_subset)
headforearm__accgyr, headforearm__accgyr_X = multivar_log_new(best_grid, headforearm_accgyr_subset)

(4847731, 8)
(4847731,)
(1211933, 8)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.74 degrees.
[[ 23518  35047  14990    270  12345     51    465  33830]
 [ 14865  78520   4132   4383   2407    205   3858  29476]
 [   901   2096  12216      0   4466      2     40    996]
 [   133   1665     45 164995     10   7314   9856     43]
 [  9480   6963  44956  12639 100924   1689  13808   7347]
 [    27   2284     14 124372      0  18663  38271     31]
 [    71   6314     30  88547      2   5687  81094    125]
 [ 23216  63003   9575    275  18476     35    367  70508]]
              precision    recall  f1-score   support

           0       0.33      0.20      0.24    120516
           1       0.40      0.57      0.47    137846
           2       0.14      0.59      0.23     20717
           3       0.42      0.90      0.57    184061
           4       0.73      0.51      0.60    197806
           5       0.55      0.10      0.17    183662
           6       0.55      0.45      0.49    181870
           7       0.50      0.38      0.43    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.74 degrees.
[[ 20459  34262  17261    369  23464     68    453  24180]
 [ 11136  83780   5388   3562   5783   1170   4340  22687]
 [  1159   1860  12471      0   3603    124    127   1373]
 [   114   1463     96 170510     22   5713   6073     70]
 [  6427   8231  33911  10828 111547   4297  13226   9339]
 [   301   2890    453 106088     24  48832  25025     49]
 [   335   5676    434  92534     29  17560  65179    123]
 [ 18053  82713  12880    201  18443     83    663  52419]]
              precision    recall  f1-score   support

           0       0.35      0.17      0.23    120516
           1       0.38      0.61      0.47    137846
           2       0.15      0.60      0.24     20717
           3       0.44      0.93      0.60    184061
           4       0.68      0.56      0.62    197806
           5       0.63      0.27      0.37    183662
           6       0.57      0.36      0.44    181870
           7       0.48      0.28      0.35    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.8 degrees.
[[ 26669  35464  13852    282  16767     55   1540  25887]
 [ 14174  75883   4023   3049   2341    426   7281  30669]
 [  1606   1330  12751      0   3528    123    267   1112]
 [   153   2257    162 172600     21   1112   7623    133]
 [ 12531   9030  41860  10443 101741   1102  14395   6704]
 [   387   8715    420 128074     27   4566  41213    260]
 [   378  12762    419  81189     26   4133  82613    350]
 [ 23643  73546   7832    189   9635     86   2920  67604]]
              precision    recall  f1-score   support

           0       0.34      0.22      0.27    120516
           1       0.35      0.55      0.43    137846
           2       0.16      0.62      0.25     20717
           3       0.44      0.94      0.60    184061
           4       0.76      0.51      0.61    197806
           5       0.39      0.02      0.05    183662
           6       0.52      0.45      0.49    181870
           7       0.51      0.36      0.42    185455

    a

### Each combo - only acc + mag

In [14]:
#create subsets for first combinations with only acc+mag
filterscolls1 = [col for col in new_subset.columns if ('thigh_acc' in col 
                                                       or 'thigh_mag' in col
                                                       or 'head_acc' in col
                                                       or 'head_mag' in col
                                                       or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                       or 'thigh_mag' in col
                                                       or 'thigh_acc' in col
                                                       or 'forearm_mag' in col 
                                                       or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                       or 'head_acc' in col
                                                       or 'head_mag' in col
                                                       or 'forearm_mag' in col
                                                       or 'Activity' in col)]
 
headthigh_accmag_subset = new_subset[filterscolls1] 
thighforearm_accmag_subset = new_subset[filterscolls2] 
headforearm_accmag_subset = new_subset[filterscolls3] 

In [15]:
#run all models for combination sensor
headthigh_accmag, headthigh_accmag_X = multivar_log_new(best_grid, headthigh_accmag_subset)
thighforearm_accmag, thighforearm_accmag_X = multivar_log_new(best_grid, thighforearm_accmag_subset)
headforearm__accmag, headforearm__accmag_X = multivar_log_new(best_grid, headforearm_accmag_subset)

(4847731, 8)
(4847731,)
(1211933, 8)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.84 degrees.
[[ 25923  22748  12096   4507  20743   3006    904  30589]
 [ 13548  56890   4905  18237   5784   3565   3538  31379]
 [  2548   1598  10127   1011   3396    660    330   1047]
 [    64    648     18  82703     67  53067  47286    208]
 [ 18208   2650  75753   6666  57123   8116  16595  12695]
 [    73   1057      1  18889     12 105658  57783    189]
 [    60   1152      1  39766     16  71782  68731    362]
 [ 28001  46764  18610   2702  23873   5234   2660  57611]]
              precision    recall  f1-score   support

           0       0.29      0.22      0.25    120516
           1       0.43      0.41      0.42    137846
           2       0.08      0.49      0.14     20717
           3       0.47      0.45      0.46    184061
           4       0.51      0.29      0.37    197806
           5       0.42      0.58      0.49    183662
           6       0.35      0.38      0.36    181870
           7       0.43      0.31      0.36    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.98 degrees.
[[ 31219  23493  11978   1783  22536   4588   3910  21009]
 [ 27063  52846   3661  11776   6967   8387   8540  18606]
 [  2083   1724  10145    120   4121   1137    101   1286]
 [   246    276     24 124216    156  12016  46975    152]
 [ 18830   4548  66973  13993  64751   2050  15483  11178]
 [   334    306     98  61045    467  16551 104666    195]
 [   441    359     50 101967    507     86  78095    365]
 [ 47352  45749  12576   6851  16917   5046   9745  41219]]
              precision    recall  f1-score   support

           0       0.24      0.26      0.25    120516
           1       0.41      0.38      0.40    137846
           2       0.10      0.49      0.16     20717
           3       0.39      0.67      0.49    184061
           4       0.56      0.33      0.41    197806
           5       0.33      0.09      0.14    183662
           6       0.29      0.43      0.35    181870
           7       0.44      0.22      0.29    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.87 degrees.
[[ 41847  26623   6692   4695  18922  12819   1752   7166]
 [ 32869  37150   1381  18842   3536  24938   5646  13484]
 [  2340   1748  10021    458   4361    549    724    516]
 [   487    978     37  59432     28  70611  52320    168]
 [ 20386   6818  65334   3746  71767  22730   4526   2499]
 [  1957   1229    132   3997     21 136747  39468    111]
 [  1276   3849    100  33066     17 100933  42457    172]
 [ 59523  45769   3053   2651  15686  30875   3666  24232]]
              precision    recall  f1-score   support

           0       0.26      0.35      0.30    120516
           1       0.30      0.27      0.28    137846
           2       0.12      0.48      0.19     20717
           3       0.47      0.32      0.38    184061
           4       0.63      0.36      0.46    197806
           5       0.34      0.74      0.47    183662
           6       0.28      0.23      0.26    181870
           7       0.50      0.13      0.21    185455

    

### Each combo - only gyr + mag

In [16]:
#create subsets for first combinations with only gyr+mag
filterscolls1 = [col for col in new_subset.columns if ('thigh_mag' in col 
                                                       or 'thigh_gyr' in col
                                                       or 'head_mag' in col
                                                       or 'head_gyr' in col
                                                       or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('forearm_mag' in col 
                                                       or 'thigh_gyr' in col
                                                       or 'thigh_mag' in col
                                                       or 'forearm_gyr' in col 
                                                       or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('forearm_mag' in col 
                                                       or 'head_mag' in col
                                                       or 'head_gyr' in col
                                                       or 'forearm_gyr' in col
                                                       or 'Activity' in col)]
 
headthigh_gyrmag_subset = new_subset[filterscolls1] 
thighforearm_gyrmag_subset = new_subset[filterscolls2] 
headforearm_gyrmag_subset = new_subset[filterscolls3] 

In [17]:
#run all models for combination sensor
headthigh_gyrmag, headthigh_gyrmag_X = multivar_log_new(best_grid, headthigh_gyrmag_subset)
thighforearm_gyrmag, thighforearm_gyrmag_X = multivar_log_new(best_grid, thighforearm_gyrmag_subset)
headforearm__gyrmag, headforearm__gyrmag_X = multivar_log_new(best_grid, headforearm_gyrmag_subset)

(4847731, 8)
(4847731,)
(1211933, 8)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.67 degrees.
[[ 20702  24408  18094   3773  26751   3634     20  23134]
 [  8570  39554  13267  16185  19598  15226     39  25407]
 [  3512   3145   4507   1404   5354    392      1   2402]
 [   148    606    168  75117     57 107799     22    144]
 [ 14256   2639  33244   6142 113162  23962    188   4213]
 [   104   2010     62  11308     10 169753    306    109]
 [   188   2571    256  50091     78 127345    683    658]
 [ 18993  40026  17714   1126  37345   6038      6  64207]]
              precision    recall  f1-score   support

           0       0.31      0.17      0.22    120516
           1       0.34      0.29      0.31    137846
           2       0.05      0.22      0.08     20717
           3       0.45      0.41      0.43    184061
           4       0.56      0.57      0.57    197806
           5       0.37      0.92      0.53    183662
           6       0.54      0.00      0.01    181870
           7       0.53      0.35      0.42    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.78 degrees.
[[ 20872  20816  24128    437  23885    633   1462  28283]
 [ 18294  55607  13559   1693  10851   3058   5037  29747]
 [  2192   2787   4627     28   5195     16    324   5548]
 [   454   1197     66  15143     32 126118  40727    324]
 [ 11909   3007  28066   1476 121547  19117   7339   5345]
 [  2209   3291     12  11490    146 151114  14895    505]
 [  3238   6496    103   5888    233 119390  45832    690]
 [ 22218  41981  36450     47  21187    314   2207  61051]]
              precision    recall  f1-score   support

           0       0.26      0.17      0.21    120516
           1       0.41      0.40      0.41    137846
           2       0.04      0.22      0.07     20717
           3       0.42      0.08      0.14    184061
           4       0.66      0.61      0.64    197806
           5       0.36      0.82      0.50    183662
           6       0.39      0.25      0.31    181870
           7       0.46      0.33      0.39    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.87 degrees.
[[ 27643  32402   6766   2609  31468   1170   5152  13306]
 [ 22753  48062   5354  10540  19673   2034   8471  20959]
 [  2681   5119   3686    779   5331    114   1242   1765]
 [   423   1060    189  89520    385  57674  34077    733]
 [ 24712  17291  15813   5092 105910   5112  16643   7233]
 [  4205   2618    451  20875   1288 109518  41615   3092]
 [  4400   8117    590  65137   2666  31790  65965   3205]
 [ 46054  51338   4165   2588  20293    462  13266  47289]]
              precision    recall  f1-score   support

           0       0.21      0.23      0.22    120516
           1       0.29      0.35      0.32    137846
           2       0.10      0.18      0.13     20717
           3       0.45      0.49      0.47    184061
           4       0.57      0.54      0.55    197806
           5       0.53      0.60      0.56    183662
           6       0.35      0.36      0.36    181870
           7       0.48      0.25      0.33    185455

    

### Each combo - only acc + gyr + mag

In [18]:
#create subsets for first combinations with only acc+gyr+mag
filterscolls1 = [col for col in new_subset.columns if ('thigh_acc' in col 
                                                       or 'thigh_gyr' in col
                                                       or 'head_acc' in col
                                                       or 'head_gyr' in col
                                                       or 'thigh_mag' in col
                                                       or 'head_mag' in col
                                                       or 'Activity' in col)]
filterscolls2 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                       or 'thigh_gyr' in col
                                                       or 'thigh_acc' in col
                                                       or 'forearm_gyr' in col
                                                       or 'thigh_mag' in col
                                                       or 'forearm_mag' in col
                                                       or 'Activity' in col)]
filterscolls3 = [col for col in new_subset.columns if ('forearm_acc' in col 
                                                       or 'head_acc' in col
                                                       or 'head_gyr' in col
                                                       or 'forearm_gyr' in col
                                                       or 'head_mag' in col
                                                       or 'forearm_mag' in col
                                                       or 'Activity' in col)]
 
headthigh_accgyrmag_subset = new_subset[filterscolls1] 
thighforearm_accgyrmag_subset = new_subset[filterscolls2] 
headforearm_accgyrmag_subset = new_subset[filterscolls3] 

In [19]:
#run all models for combination sensor
headthigh_accgyrmag, headthigh_accgyrmag_X = multivar_log_new(best_grid, headthigh_accgyrmag_subset)
thighforearm_accgyrmag, thighforearm_accgyrmag_X = multivar_log_new(best_grid, thighforearm_accgyrmag_subset)
headforearm__accgyrmag, headforearm__accgyrmag_X = multivar_log_new(best_grid, headforearm_accgyrmag_subset)

(4847731, 12)
(4847731,)
(1211933, 12)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.8 degrees.
[[20592 31954 14752  1474 19055  1661   307 30721]
 [ 9045 72314  5053 10292  6397  2314  2346 30085]
 [ 2313  2533  9408   770  4207   594    24   868]
 [   75   957    15 73981    26 66623 42285    99]
 [ 9012  3848 59633  7065 88446  6920 16051  6831]
 [   67   818     2 52699     5 96178 33793   100]
 [   70  2342     2 53816     2 63077 62300   261]
 [16636 62099 17693   504 25577  2226   892 59828]]
              precision    recall  f1-score   support

           0       0.36      0.17      0.23    120516
           1       0.41      0.52      0.46    137846
           2       0.09      0.45      0.15     20717
           3       0.37      0.40      0.38    184061
           4       0.62      0.45      0.52    197806
           5       0.40      0.52      0.45    183662
           6       0.39      0.34      0.37    181870
           7       0.46      0.32      0.38    185455

    accuracy                           0.40   1211933
   macro avg   

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.85 degrees.
[[ 35334  25245  13593    153  13497   1901    457  30336]
 [ 19443  64936   4171   5534   4497   3589   2424  33252]
 [  1898   2188  10037      8   3423    535     20   2608]
 [   196   1316     58 114915      8  15056  52354    158]
 [ 20590   3958  45141   6440  91476   3995  18158   8048]
 [   527   3133    228  48089     14  91025  40519    127]
 [   793   5890    198  75856      6  28383  70402    342]
 [ 33891  63202  14119    251   8443    528   1099  63922]]
              precision    recall  f1-score   support

           0       0.31      0.29      0.30    120516
           1       0.38      0.47      0.42    137846
           2       0.11      0.48      0.19     20717
           3       0.46      0.62      0.53    184061
           4       0.75      0.46      0.57    197806
           5       0.63      0.50      0.55    183662
           6       0.38      0.39      0.38    181870
           7       0.46      0.34      0.39    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.82 degrees.
[[ 33295  27514  14475   1973  18839   3159   1158  20103]
 [ 22179  48827   4123  11148   3692   6141   2360  39376]
 [  1839   1930  10864    466   4048    203    641    726]
 [   381   1423     91  84764     27  66030  30822    523]
 [ 14429   7613  56317   5806  90252  16957   3672   2760]
 [  2529   3152    222  16523     26 124679  33879   2652]
 [  2596  10038    267  69937     42  69389  28383   1218]
 [ 37419  47119  10799   1298  11338   4546   2442  70494]]
              precision    recall  f1-score   support

           0       0.29      0.28      0.28    120516
           1       0.33      0.35      0.34    137846
           2       0.11      0.52      0.18     20717
           3       0.44      0.46      0.45    184061
           4       0.70      0.46      0.55    197806
           5       0.43      0.68      0.53    183662
           6       0.27      0.16      0.20    181870
           7       0.51      0.38      0.44    185455

    

## Difference between Nr positions - Subq 3
We have looked into single and double positions, what about third positions <br>

Only combine top 3 positions from SQ1 that show promising results. or reflect on SQ combinations. 

Head = smartglasses <br>
forearm = smartwatch <br>
thigh = pocket smartwatch <br>

Combinations: <br>
Head - Forearm - Thigh <br>


Different Sensors: <br>
acc (mean, sd) <br>
gyr (mean, sd) <br>
mag (mean, sd) <br>
acc + mag (mean, sd) <br>
acc + gyr (mean, sd) <br>
gyr + mag (mean, sd) <br>
acc + gyr + mag (mean, sd) <br>

In [20]:
#create subsets for combinations with only head-forearm-thigh
#only acc
filterscolls1 = [col for col in new_subset.columns if ('thigh_acc' in col 
                                                       or 'head_acc' in col
                                                       or 'forearm_acc' in col
                                                       or 'Activity' in col)]
#only gyr
filterscolls2 = [col for col in new_subset.columns if ('thigh_gyr' in col
                                                       or 'head_gyr' in col
                                                       or 'forearm_gyr' in col
                                                       or 'Activity' in col)]
#only mag
filterscolls3 = [col for col in new_subset.columns if ('thigh_mag' in col
                                                       or 'head_mag' in col
                                                       or 'forearm_mag' in col
                                                       or 'Activity' in col)]
#acc+gyr
filterscolls4 = [col for col in new_subset.columns if ('thigh_acc' in col
                                                       or 'head_acc' in col
                                                       or 'forearm_acc' in col
                                                       or 'thigh_gyr' in col
                                                       or 'head_gyr' in col
                                                       or 'forearm_gyr' in col
                                                       or 'Activity' in col)]
#acc+mag
filterscolls5 = [col for col in new_subset.columns if ('thigh_acc' in col
                                                       or 'head_acc' in col
                                                       or 'forearm_acc' in col
                                                       or 'thigh_mag' in col
                                                       or 'head_mag' in col
                                                       or 'forearm_mag' in col
                                                       or 'Activity' in col)]
#gyr+mag
filterscolls6 = [col for col in new_subset.columns if ('thigh_gyr' in col
                                                       or 'head_gyr' in col
                                                       or 'forearm_gyr' in col
                                                       or 'thigh_mag' in col
                                                       or 'head_mag' in col
                                                       or 'forearm_mag' in col
                                                       or 'Activity' in col)]
#acc+gyr+mag
filterscolls7 = [col for col in new_subset.columns if ('thigh_acc' in col
                                                       or 'head_acc' in col
                                                       or 'forearm_acc' in col
                                                       or 'thigh_gyr' in col
                                                       or 'head_gyr' in col
                                                       or 'forearm_gyr' in col
                                                       or 'thigh_mag' in col
                                                       or 'head_mag' in col
                                                       or 'forearm_mag' in col
                                                       or 'Activity' in col)]
 
thighheadforearm_acc_subset = new_subset[filterscolls1] 
thighheadforearm_gyr_subset = new_subset[filterscolls2] 
thighheadforearm_mag_subset = new_subset[filterscolls3] 
thighheadforearm_accgyr_subset = new_subset[filterscolls4] 
thighheadforearm_accmag_subset = new_subset[filterscolls5] 
thighheadforearm_gyrmag_subset = new_subset[filterscolls6]
thighheadforearm_accgyrmag_subset = new_subset[filterscolls7]

In [21]:
#run all models for combination sensor only acc, gyr, mag
thighheadforearm_acc, thighheadforearm_acc_X = multivar_log_new(best_grid, thighheadforearm_acc_subset)
thighheadforearm_gyr, thighheadforearm_gyr_X = multivar_log_new(best_grid, thighheadforearm_gyr_subset)
thighheadforearm_mag, thighheadforearm_mag_X = multivar_log_new(best_grid, thighheadforearm_mag_subset)

(4847731, 6)
(4847731,)
(1211933, 6)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.78 degrees.
[[ 24427  22273  10886    104  17908    116    941  43861]
 [ 16202  56408   1881   1752   2931   1168   8538  48966]
 [  1101   1308  11021     26   5743     71     30   1417]
 [   283   1387     15 135040     20    658  46302    356]
 [  6650   4521  51483   2440  95353    907  24734  11718]
 [   711   3805     69  60030     31  17406 101439    171]
 [   750   4581     21  33251     29   2925 139944    369]
 [ 26433  52212   7959     64  14752    268   1646  82121]]
              precision    recall  f1-score   support

           0       0.32      0.20      0.25    120516
           1       0.39      0.41      0.40    137846
           2       0.13      0.53      0.21     20717
           3       0.58      0.73      0.65    184061
           4       0.70      0.48      0.57    197806
           5       0.74      0.09      0.17    183662
           6       0.43      0.77      0.55    181870
           7       0.43      0.44      0.44    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.67 degrees.
[[ 22708  31085  20513    248  12836    100   1459  31567]
 [ 14798  66201   7959   2838   3370   2173   5996  34511]
 [  1698   1830  11767      0   2667      4    181   2570]
 [    73   1088    106 166950     27  12441   3201    175]
 [ 16236   6775  36035   8130  97729   9185  11578  12138]
 [   315   1074    184 113201     11  43002  25710    165]
 [   680   4864    571  62485     43  50716  61201   1310]
 [ 19295  53346  16224    154   9140    127   1060  86109]]
              precision    recall  f1-score   support

           0       0.30      0.19      0.23    120516
           1       0.40      0.48      0.44    137846
           2       0.13      0.57      0.21     20717
           3       0.47      0.91      0.62    184061
           4       0.78      0.49      0.60    197806
           5       0.37      0.23      0.29    183662
           6       0.55      0.34      0.42    181870
           7       0.51      0.46      0.49    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 2.02 degrees.
[[  1969  14954    842  17190  25502   5190  25420  29449]
 [  2633  27011     98  30962  21752    865  22147  32378]
 [   871    707     26   4388   4067    912   5111   4635]
 [   405    104     73  64656   7371    186 110585    681]
 [  5067   9688    115  29100  46824   1161  46588  59263]
 [   304     82   3357   9606   9499  32018 127716   1080]
 [  1376    214    114  36921   8530  25405 107527   1783]
 [  4030  19834    275  18678  31129   4175  38420  68914]]
              precision    recall  f1-score   support

           0       0.12      0.02      0.03    120516
           1       0.37      0.20      0.26    137846
           2       0.01      0.00      0.00     20717
           3       0.31      0.35      0.33    184061
           4       0.30      0.24      0.27    197806
           5       0.46      0.17      0.25    183662
           6       0.22      0.59      0.32    181870
           7       0.35      0.37      0.36    185455

    

In [22]:
#run all models for combination sensor only acc+gyr, acc+mag, gyr+mag
thighheadforearm_accgyr, thighheadforearm_accgyr_X = multivar_log_new(best_grid, thighheadforearm_accgyr_subset)
thighheadforearm_accmag, thighheadforearm_accmag_X = multivar_log_new(best_grid, thighheadforearm_accmag_subset)
thighheadforearm_gyrmag, thighheadforearm_gyrmag_X = multivar_log_new(best_grid, thighheadforearm_gyrmag_subset)

(4847731, 12)
(4847731,)
(1211933, 12)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.7 degrees.
[[ 30209  31454  12144    306  15990     54    414  29945]
 [ 16598  77701   3085   3206   2210    956   4224  29866]
 [  1655   1545  12677      4   3643     67     35   1091]
 [   238   1580     48 172480      7   3780   5880     48]
 [  8470   6215  37875  11003 109053   3516  13436   8238]
 [   507   3484    175 114910      2  34487  30084     13]
 [   569   6615    110  87078      0  15444  71953    101]
 [ 24848  66749   8247    200  14460     48    482  70421]]
              precision    recall  f1-score   support

           0       0.36      0.25      0.30    120516
           1       0.40      0.56      0.47    137846
           2       0.17      0.61      0.27     20717
           3       0.44      0.94      0.60    184061
           4       0.75      0.55      0.64    197806
           5       0.59      0.19      0.29    183662
           6       0.57      0.40      0.47    181870
           7       0.50      0.38      0.43    185455

    a

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.89 degrees.
[[ 28838  21159  11889   2835  20582   1206    986  33021]
 [ 19132  51353   2142  15796   5658   3294   3162  37309]
 [  1725   1843  10533    359   4947     39    171   1100]
 [   319    453     15  60272     46  12507 110126    323]
 [  9852   2065  73169   5421  75689   1415  23812   6383]
 [  1162    842     46   6267     97  65773 109265    210]
 [   721   1236     25  32798    116  30500 115778    696]
 [ 38734  45746  10327   1873  18129   1625   4157  64864]]
              precision    recall  f1-score   support

           0       0.29      0.24      0.26    120516
           1       0.41      0.37      0.39    137846
           2       0.10      0.51      0.16     20717
           3       0.48      0.33      0.39    184061
           4       0.60      0.38      0.47    197806
           5       0.57      0.36      0.44    183662
           6       0.32      0.64      0.42    181870
           7       0.45      0.35      0.39    185455

    

C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.83 degrees.
[[ 32251  23536  20211   1060  23630   1546   1073  17209]
 [ 22247  59703  16888   7467  10426   2445   1390  17280]
 [  3185   4040   4492    738   5334    112    273   2543]
 [   481   1339    109  98020     31  12339  71641    101]
 [ 15756   4030  25621   6965 122407   1538  18986   2503]
 [  2385   3735     15  24392    118  41572 111364     81]
 [  3409   7161     93  76579    216  25848  68199    365]
 [ 39470  48233  23301    238  21288    295   1732  50898]]
              precision    recall  f1-score   support

           0       0.27      0.27      0.27    120516
           1       0.39      0.43      0.41    137846
           2       0.05      0.22      0.08     20717
           3       0.45      0.53      0.49    184061
           4       0.67      0.62      0.64    197806
           5       0.49      0.23      0.31    183662
           6       0.25      0.37      0.30    181870
           7       0.56      0.27      0.37    185455

    

In [23]:
#run all models for combination sensor only acc+gyr, acc+mag, gyr+mag
thighheadforearm_accgyrmag, thighheadforearm_accgyrmag_X = multivar_log_new(best_grid, thighheadforearm_accgyrmag_subset)

(4847731, 18)
(4847731,)
(1211933, 18)
(1211933,)


C:\Users\31657\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean Absolute Error: 1.81 degrees.
[[ 41309  21849  13151    596  14089   1023    185  28314]
 [ 24060  60193   2577   7382   3119   3472    725  36318]
 [  2103   1907  11449    178   3725     41     61   1253]
 [   300   1263     29  68669      4  24561  89070    165]
 [  9974   3930  56800   4901  94899   6712  16630   3960]
 [  1684   2282    106   9691      4 108612  61146    137]
 [  1425   5552     63  43618      2  38086  92652    472]
 [ 45754  52847  11667     92  11172    497    548  62878]]
              precision    recall  f1-score   support

           0       0.33      0.34      0.33    120516
           1       0.40      0.44      0.42    137846
           2       0.12      0.55      0.20     20717
           3       0.51      0.37      0.43    184061
           4       0.75      0.48      0.58    197806
           5       0.59      0.59      0.59    183662
           6       0.35      0.51      0.42    181870
           7       0.47      0.34      0.39    185455

    